In [1]:
#import ipywidgets
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, Dropdown, IntSlider
from IPython.display import clear_output

In [2]:
url = 'https://drive.google.com/file/d/1kM__riNHRPx5GsyuOH3yhiql3OZvwmuP/view?usp=sharing'
path_tidy = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
df = pd.read_csv(path_tidy)

In [12]:
df.head()

,,"<!DOCTYPE html><html><head><title>Google Drive - Virus scan warning</title><meta http-equiv=""content-type"" content=""text/html; charset=utf-8""/><style nonce=""jyiNdOMesQllkQnc0RS3xA"">/* Copyright 2022 Google Inc. All Rights Reserved. */"
.goog-inline-block{position:relative;display:-moz-inline-box;display:inline-block}* html .goog-inline-block{display:inline}*:first-child+html .goog-inline-block{display:inline}.goog-link-button{position:relative;color:#15c;text-decoration:underline;cursor:pointer}.goog-link-button-disabled{color:#ccc;text-decoration:none;cursor:default}body{color:#222;font:normal 13px/1.4 arial,sans-serif;margin:0}.grecaptcha-badge{visibility:hidden}.uc-main{padding-top:50px;text-align:center}#uc-dl-icon{display:inline-block;margin-top:16px;padding-right:1em;vertical-align:top}#uc-text{display:inline-block;max-width:68ex;text-align:left}.uc-error-caption,.uc-warning-caption{color:#222;font-size:16px}...


In [4]:
#SET DEFAULT VALUES
season =  20162017
seasonW = Dropdown(options = df['season'].unique(), value = season)

game_type = 'regular_season'
game_typeW = Dropdown(options=['playoffs','regular_season'], value = game_type)
 
games = df[(df['season']==season) & (df['type_season']==game_type)]['id_game'].unique()
gameW = IntSlider(value = 0,min=0,max=len(games)-1,continuous_update=False)

events = df[(df['season']==season) & (df['type_season']==game_type) & (df['id_game']==games[0])]
eventW = IntSlider(value = 0,min=0,max=len(events)-1,continuous_update=False)

#INTERACTIF DEBUGGER
@interact(season = seasonW,game_type = game_typeW,game = gameW,event=eventW)
def debugger(season,game_type,game,event):
    
    #RESET GAMES
    games = df[(df['season']==season) & (df['type_season']==game_type)]['id_game'].unique()
    gameW.max = len(games) - 1
    
    if game > len(games):
        game = 0
        gameW.value = 0
        
    #RESET EVENTS
    events = df[(df['season']==season) & (df['type_season']==game_type) & (df['id_game']==games[game])]
    eventW.max =  len(events) - 1
    
    if event >  len(events):
        event = 0
        eventW.value = 0
        
    #GET INFO FOR THIS PARTICULAR EVENT
    event_infos = events.iloc[event]
    
    #PLOT BACKGROUND IMAGE
    fig, ax = plt.subplots()
    img = plt.imread("../figures/nhl_rink.png")
    ax.imshow(img, extent=[-100, 100, -43, 43])
    
    #SUPTITLE
    season_title = f'{str(season)[:4]}-{str(season)[4:]}'
    home = event_infos['team_home_name']
    away = event_infos['team_away_name']
   
    plt.suptitle(f'{season_title}\n\n{home} VS {away}')
    
    #TITLE
    shot_type =  event_infos['shot_type']
    event_type =  event_infos['result_event']
    player =  event_infos['shooter_name']
     
    plt.title(f'A {shot_type} of {player}: {event_type}')
   
   
    #PLOT POINT AT THE LOCATION OF THE EVENT
    x =  event_infos['x_coord']
    y =  event_infos['y_coord']
    ax.scatter(x, y, marker='o', color='r', label='point')
    
    #REMOVE PAST PLOTS IF ANY
    clear_output(wait=True)

interactive(children=(Dropdown(description='season', index=4, options=(20172018, 20192020, 20202021, 20182019,…